## Libraries 

In [ ]:
import os
import sys
# Repository source: https://github.com/qubvel/efficientnet
import tensorflow 
from tensorflow.python.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,Model

import glob
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model


import re
from tensorflow.keras.backend import argmax
from tensorflow.keras.models  Sequential

import numpy as np
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import time
import os.path

#metrices 
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


## Arcface Loss

In [ ]:
import os
import sys
import math as m
import tensorflow 
from tensorflow.python.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
#reused code from the original paper: https://github.com/deepinsight/insightface
def get_logits(labels, y_pred):
    s = 64.
    cos_t = y_pred
    sin_m = tf.math.sin(0.5)
    cos_m = tf.math.cos(0.5)
    cos_t2 = tf.square(cos_t, name='cos_2')
    sin_t2 = tf.subtract(1., cos_t2, name='sin_2')
    sin_t = tf.sqrt(sin_t2, name='sin_t')
    cos_mt = s * tf.subtract(tf.multiply(cos_t, cos_m), tf.multiply(sin_t, sin_m), name='cos_mt')
    threshold = tf.math.cos(tf.constant(m.pi) - 0.5)
    cond_v = cos_t - threshold
    cond = tf.cast(tf.nn.relu(cond_v, name='if_else'), dtype=tf.bool)
    mm = sin_m * 0.5
    keep_val = s*(cos_t - mm)
    cos_mt_temp = tf.where(cond, cos_mt, keep_val)
    mask = tf.one_hot(labels, depth=2, name='one_hot_mask')
    inv_mask = tf.subtract(1., mask, name='inverse_mask')
    s_cos_t = tf.multiply(s, cos_t, name='scalar_cos_t')
    output = tf.add(tf.multiply(s_cos_t, inv_mask), tf.multiply(cos_mt_temp, mask), name='arcface_logits')
    return output
def loss(y_true, y_pred):
    labels = argmax(y_true, axis=-1)
    logits = get_logits(labels, y_pred)
    #tf.keras.losses.SparseCategoricalCrossentropy()
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    return loss

## Extract the Features From the Images 

In [ ]:
Processed_datasets = {
    'train_dir': '../input/capstone-dataset/base__dir/train_dir',
    'val_dir': '../input/capstone-dataset/base__dir/val_dir',
    'test_dir':'../input/capstone-dataset/base__dir/test_dir'
    
}


In [ ]:
df_train = pd.read_csv("../input/capstone-dataset/base__dir/train_data.csv")
df_val = pd.read_csv("../input/capstone-dataset/base__dir/val_data.csv")
df_test = pd.read_csv("../input/capstone-dataset/base__dir/test_data.csv")

In [ ]:
num_train_samples = len(df_train)
num_val_samples = len(df_val)
num_test_samples = len(df_test)

train_batch_size = 10
val_batch_size = 10
test_batch_size = 10

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)
test_steps = np.ceil(num_test_samples / test_batch_size)

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = datagen.flow_from_directory(Processed_datasets['train_dir'],
                                        target_size=(299,299),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(Processed_datasets['val_dir'],
                                        target_size=(299,299),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

In [ ]:
effnet = EfficientNetB3(weights = "imagenet", include_top = False, input_shape=(299,299,3), pooling = "avg", classes = 2)

for layer in effnet.layers:
    layer.trainable = True
x = effnet.output
droup_out = Dropout(0.3)(x)
predicted = Dense(2,activation ='softmax')(droup_out)
model = Model(effnet.input, predicted)

model.compile(loss=loss,
                      optimizer=Adam(lr=0.0001), 
                  metrics=['acc'])
print(model.summary())


In [ ]:
mode_version = "EfficientNetB3"
checkpoint = ModelCheckpoint(f'weights-{mode_version}', monitor='val_acc',
                                           save_best_only=True, save_weights_only=True, verbose=1)


In [ ]:
pretraining_EfficientNetB3 =model.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=30,callbacks =[checkpoint])

In [ ]:
data_full = pd.read_csv("../input/data-files/Data/data_full.csv")

In [ ]:
data_with_nframes = pd.read_csv("../input/data-files/Data/only_videos_with_labels.csv")

In [ ]:
from keras.preprocessing.image import img_to_array, load_img
import numpy as np

def process_image(image, target_shape):
    Given an image, process it and return the array.
    # Load the image.
    h, w, _ = target_shape
    print(image)
    image = load_img(image, target_size=(h, w))

    # Turn it into numpy, normalize and return.
    img_arr = img_to_array(image)
    x = (img_arr / 255.).astype(np.float32)

    return x

In [ ]:
import csv
import numpy as np
import random
import glob
import os.path
import sys
import operator
import threading

from keras.utils import to_categorical

class threadsafe_iterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.iterator)

def threadsafe_generator(func):
    """Decorator"""
    def gen(*a, **kw):
        return threadsafe_iterator(func(*a, **kw))
    return gen

class DataSet():

    def __init__(self, seq_length=40, class_limit=None, image_shape=(224, 224, 3)):
        """Constructor.
        seq_length = (int) the number of frames to consider
        class_limit = (int) number of classes to limit the data to.
            None = no limit.
        """
        self.seq_length = seq_length
        self.class_limit = class_limit
        self.sequence_path = os.path.join('data', 'sequences')
        self.max_frames = 300  # max number of frames a video can have for us to use it

        # Get the data.
        self.data = self.get_data()

        # Get the classes.
        self.classes = self.get_classes()

        # Now do some minor data cleaning.
        #self.data = self.clean_data()

        self.image_shape = image_shape
    @staticmethod
    def get_data():
        """Load our data from file."""
        with open(os.path.join('../input/data-files/Data/', 'only_videos_with_labels.csv'), 'r') as fin:
            reader = csv.reader(fin)
            data = list(reader)

        return data

    def clean_data(self):
        """Limit samples to greater than the sequence length and fewer
        than N frames. Also limit it to classes we want to use."""
        data_clean = []
        for item in self.data:
            if int(item[1]) >= self.seq_length and int(item[1]) <= self.max_frames \
                    and item[2] in self.classes:
                data_clean.append(item)

        return data_clean

    def get_classes(self):
        """Extract the classes from our data. If we want to limit them,
        only return the classes we need."""
        classes = []
        for i in range (1, len(self.data)):
            item = self.data[i]
            if item[2] not in classes:
                classes.append(item[2])
        # Sort them.
        classes = sorted(classes)
        # Return.
        if self.class_limit is not None:
            return classes[:self.class_limit]
        else:
            return classes

    def get_class_one_hot(self, class_str):
        """Given a class as a string, return its number in the classes
        list. This lets us encode and one-hot it for training."""
        # Encode it first.
        #label_encoded = self.classes.index(class_str)

        # Now one-hot it.
        #label_hot = to_categorical(class_str, len(self.classes))
        label_hot = float(class_str)
        #assert len(label_hot) == len(self.classes)

        return label_hot

    def split_train_test(self):
        """Split the data into train and test groups."""
        train = []
        test = []
        valid = []
        for i in range(1,len(self.data)):
            item = self.data[i]
            if item[3] == 'Train':
                train.append(item)
            elif item[3] == 'Valid':
                valid.append(item)
            else:
                test.append(item)
        return train, valid, test

    def get_all_sequences_in_memory(self, train_test, data_type):
        """
        This is a mirror of our generator, but attempts to load everything into
        memory so we can train way faster.
        """
        # Get the right dataset.
        train, valid, test = self.split_train_test()
        if train_test == 'train':
            data = train 
        elif train_test == 'valid':
            data = valid
        else: 
            data = test

        print("Loading %d samples into memory for %sing." % (len(data), train_test))

        X, y = [], []
        for row in data:

            if data_type == 'images':
                frames = self.get_frames_for_sample(row)
                frames = self.rescale_list(frames, self.seq_length)

                # Build the image sequence
                sequence = self.build_image_sequence(frames)

            else:
                sequence = self.get_extracted_sequence(data_type, row)

                if sequence is None:
                    print("Can't find sequence. Did you generate them?")
                    raise

            X.append(sequence)
            y.append(row[2])

        return np.array(X), np.array(y)

    @threadsafe_generator
    def frame_generator(self, batch_size, train_test, data_type):
        """Return a generator that we can use to train on. There are
        a couple different things we can return:

        data_type: 'features', 'images'
        """
        # Get the right dataset for the generator.
        train, valid, test = self.split_train_test()
        if train_test == 'train':
            data = train 
        elif train_test == 'valid':
            data = valid
        else: 
            data = test

        print("Creating %s generator with %d samples." % (train_test, len(data)))

        while 1:
            X, y = [], []

            # Generate batch_size samples.
            for _ in range(batch_size):
                # Reset to be safe.
                sequence = None

                # Get a random sample.
                sample = random.choice(data)

                # Check to see if we've already saved this sequence.
                if data_type is "images":
                    # Get and resample frames.
                    frames = self.get_frames_for_sample(sample)
                    frames = self.rescale_list(frames, self.seq_length)

                    # Build the image sequence
                    sequence = self.build_image_sequence(frames)
                else:
                    # Get the sequence from disk.
                    sequence = self.get_extracted_sequence(data_type, sample)

                    if sequence is None:
                        raise ValueError("Can't find sequence. Did you generate them?")

                X.append(sequence)
                y.append(self.get_class_one_hot(sample[2]))

            yield np.array(X), np.array(y)

    def build_image_sequence(self, frames):
        """Given a set of frames (filenames), build our sequence."""
        return [process_image(x, self.image_shape) for x in frames]

    def get_extracted_sequence(self, data_type, sample):
        """Get the saved extracted features."""
        filename = sample[0]
        #data_path = "../input/sequences/"
        data_path = "../input/sequences/"
        path = os.path.join(data_path, "sequences",filename + '-' + str(self.seq_length) + \
            '-' + data_type + '.npy')
        if os.path.isfile(path):
            return np.load(path)
        else:
            return None

    def get_frames_by_filename(self, filename, data_type):
        """Given a filename for one of our samples, return the data
        the model needs to make predictions."""
        # First, find the sample row.
        sample = None
        for row in self.data:
            if row[0] == filename:
                sample = row
                break
        if sample is None:
            raise ValueError("Couldn't find sample: %s" % filename)

        if data_type == "images":
            # Get and resample frames.
            frames = self.get_frames_for_sample(sample)
            frames = self.rescale_list(frames, self.seq_length)
            # Build the image sequence
            sequence = self.build_image_sequence(frames)
        else:
            # Get the sequence from disk.
            sequence = self.get_extracted_sequence(data_type, sample)

            if sequence is None:
                raise ValueError("Can't find sequence. Did you generate them?")

        return sequence

    @staticmethod
    def get_frames_for_sample(sample):
        sample = sample[0]
        sources = []
        images = data_full.loc[data_full['Video_ID'] == sample]['Frame_ID']
        target = data_full.loc[data_full['Video_ID'] == sample]['Label']
        if target.any() == 0:
            for i in images:
                src = os.path.join(training_dataset['Real'], i)
                sources.append(src)
        else: 
            for i in images:
                src = os.path.join(training_dataset['Fake'], i)
                sources.append(src)
        return sources

    @staticmethod
    def get_filename_from_image(filename):
        parts = filename.split(os.path.sep)
        return parts[-1].replace('.jpg', '')

    @staticmethod
    def rescale_list(input_list, size):
        """Given a list and a size, return a rescaled/samples list. For example,
        if we want a list of size 5 and we have a list of size 25, return a new
        list of size five which is every 5th element of the origina list."""
        assert len(input_list) >= size

        # Get the number to skip between iterations.
        skip = len(input_list) // size

        # Build our new output.
        output = [input_list[i] for i in range(0, len(input_list), skip)]

        # Cut off the last one if needed.
        return output[:size]

    def print_class_from_prediction(self, predictions, nb_to_return=5):
        """Given a prediction, print the top classes."""
        # Get the prediction for each label.
        label_predictions = {}
        for i, label in enumerate(self.classes):
            label_predictions[label] = predictions[i]

        # Now sort them.
        sorted_lps = sorted(
            label_predictions.items(),
            key=operator.itemgetter(1),
            reverse=True
        )

        # And return the top N.
        for i, class_prediction in enumerate(sorted_lps):
            if i > nb_to_return - 1 or class_prediction[1] == 0.0:
                break
            print("%s: %.2f" % (class_prediction[0], class_prediction[1]))


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
import numpy as np


In [ ]:
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization, Flatten

from tensorflow.keras import initializers, regularizers, constraints

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys

class ResearchModels():
    def __init__(self, nb_classes, model, seq_length,
                 saved_model=None, features_length=2304):
        # Set defaults.
        self.seq_length = seq_length
        self.load_model = load_model
        self.saved_model = saved_model
        self.nb_classes = nb_classes
        self.feature_queue = deque()

        # Set the metrics. Only use top k if there's a need.
        metrics = ['accuracy']
        # Get the appropriate model.
        if self.saved_model is not None:
            print("Loading model %s" % self.saved_model)
            self.model = load_model(self.saved_model)
        elif model == 'gru':
            print("Loading gru model.")
            self.input_shape = (seq_length, features_length)
            self.model = self.gru()
        # Now compile the network.
        optimizer=Adam(lr=0.001, decay=1e-6)
        self.model.compile(loss='binary_crossentropy', optimizer=optimizer,
                           metrics=metrics)

        print(self.model.summary())

    def gru(self):
        model = Sequential()
        model.add(BatchNormalization(input_shape=self.input_shape))
        model.add(GRU(256, activation='relu',return_sequences=True))
        model.add(Dropout(0.3))

        #model.add(Bidirectional(GRU(128, activation='relu',return_sequences=True)))
        #model.add(Dropout(0.3))
        #model.add(Bidirectional(GRU(64, activation='relu',return_sequences=True)))
        #model.add(Dropout(0.3))
        #model.add(BatchNormalization())
        model.add(GRU(128,activation='relu', return_sequences=False))
        #model.add(GRU(512, return_sequences=False))
        #model.add(Bidirectional(GRU(128, dropout=0.1, recurrent_dropout=0.1, activation='relu', return_sequences=True)))
        #model.add(Bidirectional(GRU(128, dropout=0.1, recurrent_dropout=0.1, activation='relu', return_sequences=True)))
        #model.add(Flatten())
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1,activation='sigmoid'))
        return model


In [ ]:
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import time
import os.path

def train(data_type, seq_length, model, saved_model=None,
          class_limit=None, image_shape=None,
          load_to_memory=False, batch_size=10, nb_epoch=100):
    # Helper: Save the model.
    mode_version = "GRU"
    checkpoint = ModelCheckpoint(f'weights-{mode_version}.h5', monitor='val_accuracy',
                                           save_best_only=True, save_weights_only=True, verbose=1)

    # Helper: TensorBoard
    #tb = TensorBoard(log_dir=os.path.join('data', 'logs', model))

    # Helper: Stop when we stop learning.
    early_stopper = EarlyStopping(monitor ="val_accuracy", patience=5)
    
    # Helper: Save results.

    # Get the data and process it.
    if image_shape is None:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit
        )
    else:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit,
            image_shape=image_shape
        )

    # Get samples per epoch.
    # Multiply by 0.7 to attempt to guess how much of data.data is the train set.
    steps_per_epoch =  3600// batch_size
    validation_steps = 1200// batch_size
    if load_to_memory:
        # Get data.
        X, y = data.get_all_sequences_in_memory('train', data_type)
        X_test, y_test = data.get_all_sequences_in_memory('test', data_type)
    else:
        # Get generators.
        generator = data.frame_generator(batch_size, 'train', data_type)
        val_generator = data.frame_generator(batch_size, 'valid', data_type)
        test_generator = data.frame_generator(batch_size, 'test', data_type)


    # Get the model.
    rm = ResearchModels(1, model, seq_length, saved_model)

    # Fit!
    if load_to_memory:
        # Use standard fit.
        rm.model.fit(
            X,
            y,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=1,
            callbacks=[early_stopper, csv_logger],
            epochs=nb_epoch)

    else:
        # Use fit generator.
        rm.model.fit_generator(
            generator,
            steps_per_epoch=steps_per_epoch,
            epochs=nb_epoch,
            verbose=1,
            callbacks=[checkpoint,early_stopper],
            validation_data=val_generator,
            validation_steps = validation_steps,
            workers=4)
        Gru_history = pd.DataFrame(rm.model.history.history)
        Gru_history.to_csv("Gru_history.history.csv")
        return (rm.model)


model = 'gru'
saved_model = None  # None or weights file
class_limit = None # int, can be 1-101 or None
seq_length = 40
load_to_memory = False  # pre-load the sequences into memory
batch_size = 32
nb_epoch = 100

if model in ['gru', 'mlp']:
    data_type = 'features'
    image_shape = None
else:
    raise ValueError("Invalid model. See train.py for options.")

GRU_model = train(data_type, seq_length, model, saved_model=saved_model,class_limit=class_limit, image_shape=image_shape,load_to_memory=False, batch_size=batch_size, nb_epoch=nb_epoch)


In [ ]:
data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit,
            image_shape=image_shape
        )
test_generator = data.frame_generator(batch_size, 'test', data_type)


In [ ]:
X_test, y_test = data.get_all_sequences_in_memory('test', data_type)
X_valid, y_valid = data.get_all_sequences_in_memory('valid', data_type)

In [ ]:
y_valid = y_valid.astype(np.int32)
y_test= y_test.astype(np.int32)

## Get the Metrices of the Model

In [ ]:

 
 
# predict probabilities for test set
yhat_probs = GRU_model.predict(X_test)
yhat_classes = GRU_model.predict_classes(X_test)

# predict crisp classes for test set
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
#yhat_classes = np.argmax(yhat_probs,axis=1)
# reduce to 1d array
#yhat_probs = yhat_probs[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Test Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print(' TestPrecision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Test Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('Test F1 score: %f' % f1)
 
# kappa

# ROC AUC
auc = roc_auc_score(y_test, yhat_probs)
print('Test ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)

In [ ]:
# predict probabilities for test set
yhat_probs = GRU_model.predict(X_valid)
yhat_classes = GRU_model.predict_classes(X_valid)

# predict crisp classes for test set
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
#yhat_classes = np.argmax(yhat_probs,axis=1)
# reduce to 1d array
#yhat_probs = yhat_probs[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_valid, yhat_classes)
print('Valid Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_valid, yhat_classes)
print('Valid Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_valid, yhat_classes)
print('Valid Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_valid, yhat_classes)
print('Valid F1 score: %f' % f1)
 
# kappa

# ROC AUC
auc = roc_auc_score(y_valid, yhat_probs)
print('Valid ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(y_valid, yhat_classes)
print(matrix)